In [6]:
import cv2
import cvzone
from cvzone.HandTrackingModule import HandDetector
import numpy as np
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw  import AudioUtilities, IAudioEndpointVolume


devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#volume.GetMute()
vol=volume.GetMasterVolumeLevel()
vRange=volume.GetVolumeRange()
minVol=vRange[0]
maxVol=vRange[1]
per=np.interp(vol,[minVol,maxVol],[0,100])

cap=cv2.VideoCapture(1)
cap.set(3, 2560)
cap.set(4, 1440)
detector=HandDetector(detectionCon=1)

while True:
    success,img=cap.read()
    img1= cv2.flip(img,1)
    lmList,img=detector.findHands(img)
    if len(lmList)==1 and detector.fingersUp(lmList[0])==[1,1,1,0,0] :
        hand=lmList[0]
        l,cursor= detector.findDistance(hand["lmList"][8], hand["lmList"][4])
        vol=np.interp(l,[50,170],[minVol,maxVol])
        volBar=np.interp(l,[50,170],[400,150])
        per=np.interp(l,[50,170],[0,100])
        volume.SetMasterVolumeLevel(vol, None)
        cv2.rectangle(img1,(50,150),(85,400),(255,0,0),3)
        cv2.rectangle(img1,(50,int(volBar)),(85,400),(255,0,0),cv2.FILLED)
        cv2.putText(img1,f'{int(per)} %',(40,450),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0))
    
    
    
    
    cv2.imshow("Image",img1)
    if cv2.waitKey(27)==27:
        cap.release()
        cv2.destroyAllWindows()
        break